In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import time

2023-12-28 16:15:08.189404: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-28 16:15:08.511819: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-28 16:15:08.520536: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-28 16:15:08.520568: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

### 1. Model Architecture

In [2]:
def createModel():
  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape=(7*7*1280,)))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=["accuracy"])

  return model



### 2. Training 

In [3]:
# Euclidean distance 
def linear_diff(np1, np2):
  np1 = np1.flatten()
  np2 = np2.flatten()

  return np1 - np2

# TODO - also try absolute difference, cosine similarity, etc.

In [4]:
features = np.load("./train_X.npy")
trainY = np.load("./train_y.npy")

print(features.shape)
trainX = []
for i in range(0, len(features)):
  trainX.append(linear_diff(features[i][0], features[i][1]))

trainX = np.array(trainX)
print(trainX.shape)
print(trainY.shape)


# Shuffle
indices = np.arange(trainX.shape[0])
np.random.shuffle(indices)
trainX = trainX[indices]  
trainY = trainY[indices]




(7020, 2, 7, 7, 1280)
(7020, 62720)
(7020,)


In [5]:
model = createModel()

model.summary()

2023-12-28 16:15:42.903142: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-28 16:15:42.903197: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-28 16:15:42.903239: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (blankk): /proc/driver/nvidia/version does not exist
2023-12-28 16:15:42.904892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                4014144   
                                                                 
 dense_1 (Dense)             (None, 8)                 520       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 4,014,673
Trainable params: 4,014,673
Non-trainable params: 0
_________________________________________________________________


In [6]:
modelID = int(time.time())
method = "flatten&eudlidian"

print("modelID: ", modelID)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=f"./logs/{method}/" + str(modelID), histogram_freq=1)

checkpoint = ModelCheckpoint("./models/" + str(modelID) + ".h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')

callbacks_list = [checkpoint, early, tensorboard_callback]

history = model.fit(trainX, trainY, batch_size=32, epochs=100, validation_split=0.2, callbacks=callbacks_list, shuffle=True)

modelID:  1703801748
Epoch 1/100
175/176 [============================>.] - ETA: 0s - loss: 1.7189 - accuracy: 0.8039
Epoch 1: val_loss improved from inf to 0.77618, saving model to ./models/1703801748.h5
176/176 [==============================] - 12s 61ms/step - loss: 1.7169 - accuracy: 0.8038 - val_loss: 0.7762 - val_accuracy: 0.8511
Epoch 2/100
176/176 [==============================] - ETA: 0s - loss: 0.4186 - accuracy: 0.9010
Epoch 2: val_loss improved from 0.77618 to 0.31665, saving model to ./models/1703801748.h5
176/176 [==============================] - 10s 56ms/step - loss: 0.4186 - accuracy: 0.9010 - val_loss: 0.3167 - val_accuracy: 0.9259
Epoch 3/100
176/176 [==============================] - ETA: 0s - loss: 0.2265 - accuracy: 0.9377
Epoch 3: val_loss improved from 0.31665 to 0.29576, saving model to ./models/1703801748.h5
176/176 [==============================] - 10s 57ms/step - loss: 0.2265 - accuracy: 0.9377 - val_loss: 0.2958 - val_accuracy: 0.9402
Epoch 4/100
175/176 

### Testing

In [8]:
# Load best model
model = keras.models.load_model("./models/" + str(modelID) + ".h5")

In [9]:
textY = np.load("./test_y.npy") 
textX = []
features = np.load("./test_X.npy")
for i in range(0, len(features)):
  textX.append(linear_diff(features[i][0], features[i][1]))

textX = np.array(textX)

print(textX.shape)
print(textY.shape)

score = model.evaluate(textX, textY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


(1800, 62720)
(1800,)
Test loss: 0.3440956175327301
Test accuracy: 0.9011111259460449
